# Study simulation

Maël Delem  
2/02/2024

> **Packages and setup**
>
> Down below is the code to load necessary packages used for the simulation and analysis, along with some setups for the whole document (*hover over the numbers on the far right for additional explanation of code and mechanics*).
>
> ``` r
>
> # ═══ Packages ═════════════════════════════════════════════════════════════════
>
> if (!require(librarian)) install.packages(librarian)
> ```
>
> Line 3  
> The package `librairian` eases package management with the “shelf” function, which automatically: (1) checks if a package is installed; (2) installs it if need be; (3) loads the package like the “library()” function would.
>
> Line 27  
> `reticulate` allows to translate and transfer objects and functions from R to Python and vice-versa, and was thus of primary importance for the successful use of the Python toolbox on our simulated data.
>
> Lines 35,51  
> These are personal custom color palettes meant to extend my favourite palette, the color-atypical friendly Okabe-Ito color palette. The palette originally has only eight colors, but I will need nine, then up to 30 for later graphs, so I extended it with a hand-picked selection of mine.
>
> Lines 56,70  
> These are R objects that were the results of a previous run of the simulation.
>
>     Le chargement a nécessité le package : librarian
>
> ``` r
>
> # loading
> library(librarian)     
>
> # now putting packages on our library's shelves:
> shelf(
>   # ─── data management ─────────────────
>   holodeck,       # simulating multivariate data
>   cluster,        # dissimilarity matrices
>   
>   # ─── modelling ───────────────────────
>   mclust,         # mixture clustering
>   
>   #  data visualization ──────────────
>   # palettes
>   viridis,        # colour-blind friendly palettes
>   # interactive
>   plotly,         # interactive plots
>   ggdendro,       # dendrograms
>   seriation,      # dissimilarity plots
>   webshot2,       # HTML screenshots for Word render
>   webshot,
>   
>   # ─── essential package collections ───
>   doParallel,     # parallel execution
>   easystats,      # data analysis ecosystem
>   reticulate,     # R to Python                       
>   tidyverse,      # modern R ecosystem
> )
>
> # ─── Global cosmetic theme ───
> theme_set(theme_modern(base_size = 14))  
>
> # ─── My palettes ───
> pal_okabe_ito <- c(                                                  
>   "#E69F00", "#56B4E9", "#009E73",                            
>   "#F5C710", "#0072B2", "#D55E00", "#CC79A7", "#6c0009")      
>
> # We'll need 9 colors at some point
> pal_okabe_ito_extended <- c(                                 
>   "#E69F00", "#56B4E9", "#009E73",                           
>   "#F5C710", "#0072B2", "#D55E00", "#CC79A7", "#6c0009", "#414487FF")
>
> # We'll need 30 colors at another moment
> cool_30_colors <- c(                                                   
>   "#3d51b4", "#414487FF", "#003d73", "#440154FF", "#6c0009", "#b64e4e",
>   "#D55E00", "#E69F00", "#F5C710", "#FDE725FF", "#f2bb7b", "#f1afad", "#CC79A7", 
>   "#e57774", "#7AD151FF", "#57b571", "#318a4a", "#009E73", "#22A884FF", 
>   "#2A788EFF", "#0072B2", "#2da6b5", "#56B4E9", "#889be0", "#6677e0",   
>   "#3d51b4", "#414487FF", "#003d73", "#440154FF", "#6c0009", "#b64e4e"
>   )                                                                 
>
> path = "data/"
>
> # loading result data
> df <- read_rds(paste0(path, "df.RDS"))
> df_embeds <- read_rds(paste0(path, "df_embeds.RDS"))
> # Categorical and visual embeddings
> df_embeds_categ  <- read_rds(paste0(path, "df_embeds_categ.RDS"))
> df_embeds_visual <- read_rds(paste0(path, "df_embeds_visual.RDS"))
> # Subject embeddings per sub-group
> df_embed_c_sub  <- read_rds(paste0(path, "df_embed_c_sub.RDS"))
> df_embed_cs_sub <- read_rds(paste0(path, "df_embed_cs_sub.RDS"))
> df_embed_v_sub  <- read_rds(paste0(path, "df_embed_v_sub.RDS"))
> df_embed_vs_sub <- read_rds(paste0(path, "df_embed_vs_sub.RDS"))
> # Accuracy of the unsupervised alignment (bad = not tidy data)
> df_accuracy_all_bad <- read_rds(paste0(path, "df_accuracy_all_bad.RDS"))
> df_accuracy_cat_bad <- read_rds(paste0(path, "df_accuracy_cat_bad.RDS"))
> # Coordinates of the aligned embeddings from the Python output
> coordinates_aligned_embeddings <- read_rds(paste0(path, "coordinates_aligned_embeddings.RDS")) 
> ```

## Visual-spatial-verbal model of cognitive profiles

We are going to simulate 30 participants presenting four different cognitive profiles, that I defined as, respectively, *verbal* aphantasics, *spatial* aphantasics, *spatial* phantasics, and *visual* phantasics.

To simulate these four sub-groups, we use the `holodeck` R package to generate multivariate normal distributions of scores on these three dimensions for each sub-group. For instance, verbal aphantasics have normally distributed visual imagery scores centered around a mean of 0 (normalized, so negative scores are possible), 0.4 for spatial imagery, and 0.7 for verbal style; Spatial aphantasics have means of 0 for visual, 0.75 spatial, and 0.3 for verbal; etc. The numbers are arbitrary, but have been chosen by trial-and-error to obtain a model that is both well-defined and not exaggerated.

In [ ]:

# ─── Generating hypothetical imagery values for 30 subjects ─── 

# The function takes the variance and covariance of the imagery distributions
# as arguments
generate_osv_model <- function(var, cov){
  df <- 
    tibble(group = rep(c("aph", "phant"), each = 8)) |> 
    group_by(group) |> 
    mutate(
      spatial_group = c(rep("spa_low", 4), rep("spa_high", 4)),
      vis_spa_group = paste0(group, "_", spatial_group),
      verbal_group = "verbal_low",
      verbal_group  = case_when(
        vis_spa_group == "aph_spa_low" ~ "verbal_high", 
        vis_spa_group == "phant_spa_low" ~ "verbal_mid",
        TRUE ~ verbal_group)
    ) |> 
    group_by(vis_spa_group) |> 
    # ─── visual ───
    sim_discr(
      n_vars = 1, 
      var = var, 
      cov = cov, 
      # aph_s, aph_v, phant_s, phant_v
      group_means = c(0, 0, 0.6, 0.87), 
      name = "v") |> 
    # ─── spatial ───
    sim_discr(
      n_vars = 1,  
      var = var, 
      cov = cov, 
      # aph_s, aph_v, phant_s, phant_v
      group_means = c(0.75, 0.4, 0.7, 0.3), 
      name = "s") |>
    # ─── verbal ───
    sim_discr(
      n_vars = 1,  
      var = var, 
      cov = cov, 
      # aph_s, aph_v, phant_s, phant_v
      group_means = c(0.3, 0.7, 0.3, 0.5), 
      name = "i") |>
    rename(
      visual_imagery  = v_1,
      spatial_imagery = s_1,
      verbal_profile  = i_1
      )
}

df <- generate_osv_model(0.03, 0)


### Generating “prototype” embeddings from a sphere

Proposal from [StackExchange](https://stats.stackexchange.com/questions/7977/how-to-generate-uniformly-distributed-points-on-the-surface-of-the-3-d-unit-sphe) to generate points on a sphere:

Let’s use a function to generate embeddings. We get 8 nicely distributed clusters. We’ll retrieve the centroids of each cluster, which would be the “perfect” categories of each species group (say, generated by a computational model on categorical criteria).

In [ ]:

# ─── Generating a sphere of 1000 points and 8 equally distributed clusters ─── 

generate_sphere <- function(n){
  z     <- 2*runif(n) - 1          # uniform on [-1, 1]
  theta <- 2*pi*runif(n) - pi      # uniform on [-pi, pi]
  x     <- sin(theta)*sqrt(1-z^2)  # based on angle
  y     <- cos(theta)*sqrt(1-z^2) 
  
  df <- tibble(x = x, y = y, z = z)
  
  return(df)
}

# 1000 random observations with embeddings uniformly distributed on a sphere
df_embeds <- generate_sphere(1000)

# Clustering the observations in 8 groups based on their coordinates
clusters <- Mclust(df_embeds, G = 8)

# adding the classification to the data
df_embeds <- df_embeds |> mutate(group = as.factor(clusters$classification))

# getting the centroids of each cluster
df_centroids <- 
  df_embeds |> 
  group_by(group) |> 
  summarise(
    x_centroid = mean(x),
    y_centroid = mean(y),
    z_centroid = mean(z)
  )

# adding them to the data
df_embeds_2 <- left_join(df_embeds, df_centroids, by = "group")


### Categorical model embeddings

The selection procedure for the **categorical model** will consist of selecting points that are rather *close to the centroids*. Thus, we will filter the observations of the large sets to keep only points for which the distance to the centroid is inferior to a given value. That is, points for which the Euclidean norm of the vector from the observation to the centroid:

$$d(centroid, observation) = \sqrt{(x_{c} - x_{o})^{2} + (y_{c} - y_{o})^{2} + (z_{c} - z_{o})^{2}}$$

This can be done using the function `norm(coordinates, type = "2")` in R.

In [ ]:

# ─── Selecting categorical embeddings based on distances to the centroids ───

# Function to filter points of the sphere based on the distance to the centroids
generate_embeddings <- function(df, n_embeddings, distance_quantile){
  df <- 
    df |> 
    # computing the euclidean distance to the centroids for each observation
    rowwise() |> 
    mutate(
      distance = norm(
        c((x_centroid - x), (y_centroid - y), (z_centroid - z)),
        type = "2")
      ) |> 
    # filtering by distance to the centroid by group
    group_by(group) |> 
    # selecting the X% closest (specified with "distance_quantile")
    filter(distance < quantile(distance, probs = distance_quantile)) |> 
    # selecting X random observations per cluster in these 
    # (specified with "n_embeddings")
    slice(1:n_embeddings) |> 
    select(group, x, y, z) |>
    ungroup()
}

df_embeds_categ <- generate_embeddings(df_embeds_2, 8, 0.5)


### Visual model embeddings

In the case of the **visual model**, we would like approximately evenly distributed embeddings, that could also dive *inside* the sphere, i.e. representing species that are visually close although diametrically opposed when it comes to taxonomy. To do this we could try to simulate multivariate normal distributions around the centroids[1]. This can be done with the `holodeck` package.

[1] A simpler alternative would be generating the visual embeddings with the same code as the categorical ones, selecting 8 points per cluster but much more spread out (e.g. selecting 8 among the 90% closest to the centroids, which would create more variability than the categorical one set to 60%). I chose otherwise because this wouldn’t have had points reaching *inside* the sphere.

In [ ]:

# ─── Generating visual embeddings based on multivariate distributions 
# around the categorical embeddings ───

# defining the variance and covariance of the distributions
var2 <- 0.05
cov2 <- 0

# generating multivariate distributions around the categorical 3D means
df_embeds_visual <-
  tibble(
    id = as.factor(seq(1,6400)),
    category = as.factor(rep(seq(1:64), each = 100))
  )|> 
  group_by(category) |> 
  sim_discr(
    n_vars = 1, 
    var = var2, 
    cov = cov2, 
    group_means = df_embeds_categ$x, 
    name = "x") |> 
  sim_discr(
    n_vars = 1, 
    var = var2, 
    cov = cov2, 
    group_means = df_embeds_categ$y, 
    name = "y") |> 
  sim_discr(
    n_vars = 1, 
    var = var2, 
    cov = cov2, 
    group_means = df_embeds_categ$z, 
    name = "z") |> 
  # keeping only 8 points per distribution
  slice(1) |> 
  ungroup() |> 
  mutate(group = as.factor(rep(seq(1, 8), each = 8))) |> 
  rename(x = x_1, y = y_1, z = z_1) |> 
  select(group, x, y, z)


### Generating the subject embeddings

We have four “reference” sets of embeddings which represent animals either judged according to their similarity in categorical terms (namely, species), or in visual terms (namely shape or color similarities, assuming that these similarities are more evenly distributed, e.g. the crab looks like a spider, but is also pretty close to a scorpion, etc.).

To generate the embeddings of each subject in each condition, we will start from these reference embeddings and generate random noise around *each item*, i.e. for all 64 animals. For 100 subjects, we would thus generate 100 noisy points around each animal, each point corresponding to a given subject.

The visual and verbal groups will be generated with slightly more intra-group variance, so as to try to make the spatial groups as coherent as possible (and avoid blurring everything and making the groups disappear in noise).

In [ ]:

# ─── Generating unique embeddings for each subject based on the models ───

# creating dfs with participants
df_subjects_7 <- 
  tibble(subject = seq(1, 7, 1)) |> 
  mutate(subject = paste0("subject_", subject))

df_subjects_8 <- 
  tibble(subject = seq(1, 8, 1)) |> 
  mutate(subject = paste0("subject_", subject))

# splitting df_embeddings
df_embed_c  <- df_embeddings |> select(group, species,  x_c:z_c)
df_embed_cs <- df_embeddings |> select(group, species, x_cs:z_cs)
df_embed_vs <- df_embeddings |> select(group, species, x_vs:z_vs)
df_embed_v  <- df_embeddings |> select(group, species,  x_v:z_v)

# function to create embeddings per subject with normal random noise
generate_subject_embeddings <- function(df, df_subjects, var){
  df <-
    df |> 
    mutate(subject = list(df_subjects)) |> 
    unnest(subject) |> 
    group_by(species) |> 
    # simulating x coordinates
    sim_discr(
      n_vars = 1, 
      var = var, 
      cov = 0, 
      group_means = pull(df, 3), 
      name = "x") |> 
    # simulating y coordinates
    sim_discr(
      n_vars = 1, 
      var = var, 
      cov = 0, 
      group_means = pull(df, 4), 
      name = "y") |> 
    # simulating z coordinates
    sim_discr(
      n_vars = 1, 
      var = var, 
      cov = 0, 
      group_means = pull(df, 5), 
      name = "z") |>
  select(group, species, subject, 7:9) |> 
  rename(x = 4, y = 5, z = 6) |> 
  ungroup()
  
  return(df)
}

var_s1 = 0.001
var_s2 = 0.0005

df_embed_c_sub  <- generate_subject_embeddings(df_embed_c,  df_subjects_4, var_s1)
df_embed_cs_sub <- generate_subject_embeddings(df_embed_cs, df_subjects_4, var_s2)
df_embed_vs_sub <- generate_subject_embeddings(df_embed_vs, df_subjects_4, var_s2)
df_embed_v_sub  <- generate_subject_embeddings(df_embed_v,  df_subjects_4, var_s1)
